# Composite Indicators: International CLIs and Influence Maps

## 1 Basic settings

### 1.1 Load libraries

In [ ]:
import os
from cif import cif
import pandas as pd
import re
import datetime
from collections import Counter
import warnings
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import Image

In [ ]:
import importlib
importlib.reload(cif)

In [ ]:
%matplotlib inline

### 1.2 Check availability of X-13ARIMA-SEATS model

The model can be downloaded from https://www.census.gov/srd/www/x13as/ and its directory needs to be added to the system variables.

In [ ]:
print(os.environ['X13PATH'])

### 1.3 Settings

Change the country/countries of interest and other default settings here. For the complete list of available country codes run

```python
cif.getOECDJSONStructure(dsname = 'MEI', showValues = [0])
```

In [ ]:
#countries = ['CZE', 'AUT', 'DEU', 'POL', 'SVK'] # Select input data countries
countries = ['CZE', 'SVK'] # Select input data countries
#countries = ['AUT', 'BEL', 'BGR', 'CYP', 'CZE', 'DNK', 'EST', 'FIN', 'FRA', 'DEU', 'GRC', 'HUN', 'IRL', 'ITA', 'LVA', 'LTU', 'LUX', 'MLT', 'NLD', 'POL', 'PRT', 'ROU', 'SVK', 'SVN', 'ESP', 'SWE', 'GBR'] # Select input data countries

country = countries[0] # Select target country

#os.chdir('C:/path/') # Set path to to folder, where the plots and logs should be saved (optional)

bw = False # True for black and white visualisations

saveData = True # Save the original data sets if True

### 1.4 Output directory

In [ ]:
strDate = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")

outputDir = os.path.join('plots_' + '-'.join(countries) + '_' + strDate)
os.makedirs(outputDir, exist_ok = True)

## 2 Individual Indicators

These are the input variables, that are used to analyse and predict the cyclical component of business cycles. This part of the calculation is the same, no matter how many target countries is there.

### 2.1 Data Load

Loading data from OECD API.

In [ ]:
data_all, subjects_all, measures_all = cif.createDataFrameFromOECD(countries = countries, dsname = 'MEI', frequency = 'M')

print('Downloaded MEI data set size: %d x %d' % (data_all.shape[0], data_all.shape[1]))

In [ ]:
# Save the data

if saveData:

    data_all.to_csv(os.path.join(outputDir, 'data_all.csv'))
    subjects_all.to_csv(os.path.join(outputDir, 'subjects_all.csv'))
    measures_all.to_csv(os.path.join(outputDir, 'measures_all.csv'))

In [ ]:
data_all.tail(12) # MEI database data from last year

In [ ]:
# Leading indicators: Component series

colMultiInd = data_all.columns.names.index('subject')

ind_LOCO = subjects_all['id'].apply(lambda x: re.search(r'\bLOCO', x) != None)
subjects_LOCO = subjects_all[ind_LOCO]


# Leading indicators: Reference series

ind_LORS = subjects_all['id'].apply(lambda x: re.search(r'\bLORS', x) != None)
subjects_LORS = subjects_all[ind_LORS]


# Leading indicators: CLI

ind_LOLI = subjects_all['id'].apply(lambda x: re.search(r'\bLOLI', x) != None)
subjects_LOLI = subjects_all[ind_LOLI]


# Candidate time series

subjects_adj = subjects_all[-(ind_LOCO | ind_LORS | ind_LOLI)]
data_adj = data_all.loc[ : , [x for x in data_all.columns if x[colMultiInd] in list(subjects_adj['id'])]].copy()

### 2.2 Data Transformations

#### 2.2.1 Priority list of OECD available measures

In [ ]:
priorityList = ['NCML'
                , 'ML'
                , 'CXML'
                , 'ST'
                , 'NCCU'
                , 'CXCU'
                , 'IXOB'
                , 'NCMLSA'
                , 'MLSA'
                , 'CXMLSA'
                , 'STSA'
                , 'NCCUSA'
                , 'CXCUSA'
                , 'IXOBSA'
                , 'IXNSA'
                , 'GP'
                , 'GY']

if data_adj.shape[0] > 0:
    
    data = cif.getOnlyBestMeasure(df = data_adj, priorityList = priorityList)
    data = cif.getRidOfMultiindex(df = data)
    data = cif.getIndexAsDate(data)

#### 2.2.2 Seasonal adjustment, outlier filtering and short-term prediction & Cycle identification (Hodrick-Prescott filter) & Normalisation

In [ ]:
with warnings.catch_warnings():
            
    warnings.simplefilter("ignore")
            
    fileLogs = open(os.path.join(outputDir, 'fileLogs_dataTransformation.txt'), 'w')
    #data_SA_HP_norm = cif.pipelineTransformations(df = data, showPlots = False, savePlots = outputDir, saveLogs = fileLogs, createInverse = True)
    data_SA_HP_norm = cif.pipelineTransformations(df = data, showPlots = False, savePlots = False, saveLogs = False, createInverse = True)
    fileLogs.close()

### 2.3 Turning-point detection (Bry-Boschan algorithm)

In [ ]:
fileLogs = open(os.path.join(outputDir, 'fileLogs_dataEvaluation.txt'), 'w')
#data_ind_turningPoints = cif.pipelineTPDetection(df = data_SA_HP_norm, origColumns = list(data.columns), printDetails = False, showPlots = False, savePlots = outputDir, saveLogs = fileLogs)
data_ind_turningPoints = cif.pipelineTPDetection(df = data_SA_HP_norm, origColumns = list(data.columns), printDetails = False, showPlots = False, savePlots = False, saveLogs = False)
fileLogs.close()

In [ ]:
Image(os.path.join(outputDir, 'CZE_BCBUTE02_STSA' + '_05_ext.png'), width = 600) # change name of the series here

## 3 Reference Series

The target variable, which describes the movements of business cycle well (usually GDP).

### 3.1 Data Load

Loading data from OECD API.

In [ ]:
data_rs, subjects_rs, measures_rs = cif.createDataFrameFromOECD(countries = [country], dsname = 'QNA', subject = ['B1_GE'], frequency = 'Q')

print('Downloaded reference data set size: %d x %d' % (data_rs.shape[0], data_rs.shape[1]))

In [ ]:
# Save the data

if saveData:
    
    data_rs.to_csv(os.path.join(outputDir, 'data_rs.csv'))
    subjects_rs.to_csv(os.path.join(outputDir, 'subjects_rs.csv'))
    measures_rs.to_csv(os.path.join(outputDir, 'measures_rs.csv'))

### 3.2 Data Transformations

#### 3.2.1 Priority list of reference series (GDP) and frequency conversion

In [ ]:
rsPriorityList = [ 'LNBQRSA' # Best fit with OECD reference series
                , 'CQR'
                , 'LNBQR'
                , 'DNBSA'
                , 'DOBSA'
                , 'CQRSA'
                , 'CARSA'
                , 'GPSA'
                , 'GYSA'
                , 'CPCARSA'
                , 'VIXOBSA'
                , 'VOBARSA'
                , 'VPVOBARSA'
                , 'HCPCARSA'
                , 'HVPVOBARSA'
                ]

if (data_rs.shape[0] > 0):
    
    rsq = cif.getOnlyBestMeasure(df = data_rs, priorityList = rsPriorityList)
    rsq = cif.getRidOfMultiindex(df = rsq)
    rsq = cif.renameQuarterlyIndex(df = rsq)
    rsq = cif.getIndexAsDate(df = rsq)
    rs = cif.createMonthlySeries(df = rsq)
    rs.dropna(inplace = True)

In [ ]:
data_rs.tail(4) # all available measures of the reference series (last year, quaterly series)

In [ ]:
rs.tail(12) # selected measure of the reference series (last year, monthly series)

#### 3.2.2 Seasonal adjustment, outlier filtering and short-term prediction & Cycle identification (Hodrick-Prescott filter) & Normalisation

In [ ]:
fileLogs = open(os.path.join(outputDir, country + '_fileLogs_rsTransformation.txt'), 'w')
rs_SA_HP_norm = cif.pipelineTransformations(rs, showPlots = False, savePlots = outputDir, saveLogs = fileLogs)
fileLogs.close()

### 3.3 Turning-point detection (Bry-Boschan algorithm)

In [ ]:
fileLogs = open(os.path.join(outputDir, country + '_fileLogs_rsEvaluation.txt'), 'w')
rs_ind_turningPoints = cif.pipelineTPDetection(df = rs_SA_HP_norm, printDetails = False, showPlots = False, savePlots = outputDir, saveLogs = fileLogs)
fileLogs.close()

In [ ]:
Image(os.path.join(outputDir, 'CZE_B1_GE_LNBQRSA' + '_05_ext.png'), width = 600) # change name of the series here

## 4 Turning-points matching

In [ ]:
fileLogs = open(os.path.join(outputDir, country + '_fileLogs_tpMatching.txt'), 'w')
data_ind_extOrd, data_ind_time, data_ind_missing, data_ind_missingEarly, data_ind_extra = cif.pipelineTPMatching(df1 = rs_SA_HP_norm, df2 = data_SA_HP_norm, ind1 = rs_ind_turningPoints, ind2 = data_ind_turningPoints, printDetails = False, showPlots = False, savePlots = outputDir, saveLogs = fileLogs, nameSuffix = '_06_matching' + '_rs' + country)
fileLogs.close()

In [ ]:
Image(os.path.join(outputDir, 'CZE_BCBUTE02_STSA' + '_06_matching_rs' + country + '.png'), width = 600) # change name of the series here

## 5 Evaluation

In [ ]:
data_totalEval, data_selectedEval, data_selectedCol = cif.pipelineEvaluation(df1 = rs_SA_HP_norm, df2 = data_SA_HP_norm, missing = data_ind_missing, missingEarly = data_ind_missingEarly, extra = data_ind_extra, time = data_ind_time, maxInd = 100)

In [ ]:
data_selectedEval

In [ ]:
data_selectedEval.to_csv(os.path.join(outputDir, country + '_selectedSeries.txt'))

## 6 Aggregation & final evaluation 

### 6.1 CLI construction

In [ ]:
agg_cMat = data_SA_HP_norm.loc[:, data_selectedCol] # value of the de-trended, smoothed and normalised component

CLI = cif.pipelineCreateCLI(agg_cMat).rename(columns = {'CLI': country + '_CLI'})

In [ ]:
cif.compareTwoSeries(CLI, rs_SA_HP_norm)

### 6.2 CLI turning points

In [ ]:
fileLogs = open(os.path.join(outputDir, country + '_fileLogs_CLIEvaluation.txt'), 'w')
CLI_ind_turningPoints = cif.pipelineTPDetection(CLI, printDetails = False, showPlots = False, savePlots = outputDir, saveLogs = fileLogs)
fileLogs.close()

### 6.3 Match turning points

In [ ]:
CLI_ind_extOrd, CLI_ind_time, CLI_ind_missing, CLI_ind_missingEarly, CLI_ind_extra = cif.pipelineTPMatching(df1 = rs_SA_HP_norm, df2 = CLI, ind1 = rs_ind_turningPoints, ind2 = CLI_ind_turningPoints, showPlots = False, savePlots = outputDir, nameSuffix = '_06_matching' + '_rs' + country, bw = bw)

In [ ]:
Image(os.path.join(outputDir, 'CZE_CLI' + '_06_matching_rs' + country + '.png'), width = 600) # change name of the series here

### 6.4 Basic characteristics of created CLI

In [ ]:
CLI_eval = cif.pipelineEvaluation(df1 = rs_SA_HP_norm, df2 = CLI, missing = CLI_ind_missing, missingEarly = CLI_ind_missingEarly, extra = CLI_ind_extra, time = CLI_ind_time, evalOnly = True)

In [ ]:
CLI_eval

In [ ]:
CLI_eval.to_csv(os.path.join(outputDir, country + '_scoreCLI.txt'))

## 7 Influence maps

### 7.1 Download Shapefiles

In [ ]:
for i in countries:
    
    cif.downloadShapefile(country = i, outputDir = os.path.join('maps'))

### 7.2 Draw maps

In [ ]:
# Choose color map

# 'Greys' for black and white visualisations, other possibilities: 'Reds', 'Blues', ...
# more info: https://matplotlib.org/users/colormaps.html

if bw:
    
    colorMap = 'Greys'
    
else:
    
    colorMap = 'OrRd'# Experiment with other colors ('RdPu', 'YlOrRd', 'Reds') 
    

# Count max count (the same max for all charts)

selCountryCounter = Counter([re.split('_', x)[0] for x in data_selectedEval.index])
selCountryCount = pd.DataFrame.from_dict(selCountryCounter, orient = 'index').reset_index()
selCountryCount.rename(columns = {'index': 'from', 0: 'count'}, inplace = True)
selCountryCount['to'] = country
selCountryCount.to_csv(os.path.join(outputDir, country + '_data_influence' + '.txt'), index = False)

numColors = max(selCountryCount['count'])
normalize = mpl.colors.Normalize(vmin = 0, vmax = numColors)

cmap = mpl.cm.get_cmap(name = colorMap)


# Draw maps

for i in [100]:#range(1, data_selectedEval.shape[0] + 1):
    
    data_selectedPart = data_selectedEval[ : i]

    selCountryCounter = Counter([re.split('_', x)[0] for x in data_selectedPart.index])
    selCountryCount = pd.DataFrame.from_dict(selCountryCounter, orient = 'index').reset_index()
    selCountryCount.rename(columns = {'index': 'from', 0: 'count'}, inplace = True)
    selCountryCount['to'] = country
    selCountryCount.to_csv(os.path.join(outputDir, country + '_data_influence_' + str(i) + '.txt'), index = False)

    countryData = pd.DataFrame.from_csv(os.path.join('maps', country + '_adm_shp', country + '_adm0.csv'))
    countryFullName = countryData['NAME_ENGLISH'].iloc[0]
    
    fig = plt.figure()
    ax = plt.axes(projection = ccrs.PlateCarree())
    ax.background_patch.set_visible(False)
    ax.outline_patch.set_visible(False)
    ax.coastlines(resolution = '10m', linewidth = 0.2) # '10m', '50m', '110m'
    fig.set_facecolor('white')

    #numColors = max(selCountryCount['count'])
    #normalize = mpl.colors.Normalize(vmin = 0, vmax = numColors)

    #cmap = mpl.cm.get_cmap(name = colorMap)

    allShapes = []

    for thisCountry in countries:

        fname = os.path.join('maps', thisCountry + '_adm_shp', thisCountry + '_adm0.shp')
        countryShapes = list(shpreader.Reader(fname).geometries())
        allShapes += countryShapes

        try:

            thisCount = selCountryCount[(selCountryCount['from'] == thisCountry) & (selCountryCount['to'] == country)]['count'].iloc[0]
            thisColor = cmap(normalize(thisCount))

        except IndexError:

            thisCount = 0
            thisColor = 'white'
        
        ax.add_geometries(countryShapes,
                          ccrs.PlateCarree(),
                          edgecolor = 'black',
                          facecolor = thisColor,
                          linewidth = 0.2)

    bounds = [i.bounds for i in allShapes]
    W = min([i[0] for i in bounds])
    S = min([i[1] for i in bounds])
    E = max([i[2] for i in bounds])
    N = max([i[3] for i in bounds])

    ax.set_extent([W, E, S, N], ccrs.PlateCarree())
    ax.set_aspect(aspect = 'auto')

    plt.title('Composition of leading indicator of %s (top %d)' % (countryFullName, i), fontsize = 8)

    scalarMappable = mpl.cm.ScalarMappable(norm = normalize, cmap = cmap)
    scalarMappable.set_array(numColors)
    cbar = plt.colorbar(scalarMappable, orientation = 'horizontal', aspect = 60, ticks = list(range(numColors + 1)))
    cbar.ax.tick_params(labelsize = 8)

    fig.savefig(os.path.join(outputDir, country + '_influenceMap_max' + str(i)), dpi = 500)

    plt.show()
    plt.close()

In [ ]:
selCountryCount.sort_values('count', ascending = False)